# Resilient Distributed Datasets (RDD)

In [0]:
employees = (
    spark
    .read
    .format("csv")
    .option("header", True)
    .load("/databricks-datasets/retail-org/company_employees/")
)

## acceder au RDD sous-jacent à la classe "DataFrame"
en utilisant l'attribut .rdd:

In [0]:
type(employees.rdd)

Out[3]: pyspark.rdd.RDD

### employer les méthodes historiques associées au RDD:


In [0]:
employees.rdd.max()

Out[4]: Row(employee_id='99', employee_name='BAUM,  ELIZABETH A', department='OFFICE', region=None, employee_key='48100', active_record='1', active_record_start='2019-02-11', active_record_end=None)

In [0]:
tmp_rdd = (
    employees.rdd
    .map(
        lambda x: x["employee_name"].upper()
        )
    #.collect()
) 


### Créer directement un RDD 
sans passer par une DataFrame

In [0]:
employees_rdd =(
    spark
    .sparkContext
    .textFile("/databricks-datasets/retail-org/company_employees/")
)
type(employees_rdd)

Out[7]: pyspark.rdd.RDD

### Les propriétés d'un RDD:

In [0]:
print(f"{employees_rdd.toDebugString().decode('ascii')}")

(2) /databricks-datasets/retail-org/company_employees/ MapPartitionsRDD[27] at textFile at NativeMethodAccessorImpl.java:0 []
 |  /databricks-datasets/retail-org/company_employees/ HadoopRDD[26] at textFile at NativeMethodAccessorImpl.java:0 []


In [0]:
employees_rdd = (
    employees_rdd.distinct()
)
type(employees_rdd)

Out[9]: pyspark.rdd.PipelinedRDD

In [0]:
print(f"{employees_rdd.toDebugString().decode('ascii')}")

(2) PythonRDD[32] at RDD at PythonRDD.scala:58 []
 |  MapPartitionsRDD[31] at mapPartitions at PythonRDD.scala:205 []
 |  ShuffledRDD[30] at partitionBy at NativeMethodAccessorImpl.java:0 []
 +-(2) PairwiseRDD[29] at wrapper at <command-1081546175771657>:2 []
    |  PythonRDD[28] at wrapper at <command-1081546175771657>:2 []
    |  /databricks-datasets/retail-org/company_employees/ MapPartitionsRDD[27] at textFile at NativeMethodAccessorImpl.java:0 []
    |  /databricks-datasets/retail-org/company_employees/ HadoopRDD[26] at textFile at NativeMethodAccessorImpl.java:0 []


In [0]:
employees_rdd.getNumPartitions()


Out[11]: 2

In [0]:
# Un fonction pour avoir plus d'informations sur les partitions:
# Trouvé sur https://dev.to/aloneguid/getting-info-about-spark-partitions-4nmf

from pyspark import RDD
from pyspark.sql import DataFrame


def print_partition_info(df: DataFrame):

    import statistics

    def get_partition_len(iterator):
        yield sum(1 for _ in iterator)

    rdd: RDD = df.rdd

    count = rdd.getNumPartitions()
    # lengths = rdd.glom().map(len).collect() # much more memory hungry than next line
    lengths = rdd.mapPartitions(get_partition_len, True).collect()

    print("")
    print(f"{count} partition(s) total.")

    print(f"size stats")
    print(f"     min: {min(lengths)}")
    print(f"     max: {max(lengths)}")
    print(f"     avg: {sum(lengths)/len(lengths)}")
    print(f"  stddev: {statistics.stdev(lengths)}")

    print("")
    print("detailed info")
    for i, pl in enumerate(lengths):
        print(f"  {i}. {pl}")


In [0]:
print_partition_info(employees.repartition(3))


3 partition(s) total.
size stats
     min: 170
     max: 170
     avg: 170.0
  stddev: 0.0

detailed info
  0. 170
  1. 170
  2. 170


# pipelining: comprendre la division en tasks et stages

In [0]:
data = [1, 2, 3, 3, 4, 5, 6, 7, 8, 9]
rdd = spark.sparkContext.parallelize(data)

squared_rdd = rdd.map(lambda x: x**2)
filtered_rdd = squared_rdd.filter(lambda x: x % 2 != 0)
key_value_rdd = filtered_rdd.map(lambda x: (x, x**2))
grouped_rdd = key_value_rdd.reduceByKey(lambda x, y: x + y)  # everyday I'm shuffle-in'

result = grouped_rdd.collect()
result


Out[54]: [(1, 1), (9, 162), (25, 625), (49, 2401), (81, 6561)]

In [0]:
grouped_rdd.count()

Out[55]: 5

## Utiliser l'UI pour voir les steps

In [0]:
data = [1, 2, 3, 3, 4, 5, 6, 7, 8, 9]
rdd = spark.sparkContext.parallelize(data)

rdd = rdd.map(lambda x: x**2)
rdd = rdd.repartition(2)
rdd = rdd.filter(lambda x: x % 2 != 0)
rdd = rdd.map(lambda x: (x, x**2))
rdd = rdd.repartition(2)
rdd = rdd.reduceByKey(lambda x, y: x + y)  # wide Dependancy: everyday I'm shuffle-in'

result = rdd.collect()
result


Out[52]: [(9, 162), (25, 625), (49, 2401), (1, 1), (81, 6561)]

In [0]:
result.count()

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
File <command-3298631271999410>:1
----> 1 result.count()

TypeError: list.count() takes exactly one argument (0 given)

# Exemple with real dataset
(à suivre avec l'une des vidéos suivantes ;)

In [0]:
dbutils.fs.ls("/databricks-datasets/weather/high_temps")

Out[14]: [FileInfo(path='dbfs:/databricks-datasets/weather/high_temps', name='high_temps', size=19180, modificationTime=1540403358000)]

In [0]:
weather = (
    spark
    .read
    .format("csv")
    .option("header", True)
    .load("/databricks-datasets/weather/high_temps/")
)

In [0]:
display(weather)

date temp 2015-01-01 42 2015-01-02 42 2015-01-03 41 2015-01-04 51 2015-01-05 54 2015-01-06 54 2015-01-07 46 2015-01-08 46 2015-01-09 50 2015-01-10 46 2015-01-11 49 2015-01-12 52 2015-01-13 49 2015-01-14 43 2015-01-15 46 2015-01-16 53 2015-01-17 56 2015-01-18 57 2015-01-19 50 2015-01-20 50 2015-01-21 45 2015-01-22 49 2015-01-23 54 2015-01-24 58 2015-01-25 63 2015-01-26 61 2015-01-27 52 2015-01-28 54 2015-01-29 54 2015-01-30 47 2015-01-31 45 2015-02-01 49 2015-02-02 52 2015-02-03 50 2015-02-04 51 2015-02-05 56 2015-02-06 58 2015-02-07 54 2015-02-08 59 2015-02-09 56 2015-02-10 55 2015-02-11 55 2015-02-12 62 2015-02-13 60 2015-02-14 58 2015-02-15 54 2015-02-16 59 2015-02-17 61 2015-02-18 54 2015-02-19 51 2015-02-20 52 2015-02-21 54 2015-02-22 53 2015-02-23 55 2015-02-24 52 2015-02-25 50 2015-02-26 53 2015-02-27 50 2015-02-28 54 2015-03-01 52 2015-03-02 52 2015-03-03 51 2015-03-04 55 2015-03-05 56 2015-03-06 59 2015-03-07 62 2015-03-08 63 2015-03-09 58 2015-03-10 56 2015-03-11 58 2015-03-12 64 2015-03-13 63 2015-03-14 57 2015-03-15 51 2015-03-16 57 2015-03-17 56 2015-03-18 60 2015-03-19 60 2015-03-20 57 2015-03-21 56 2015-03-22 53 2015-03-23 52 2015-03-24 55 2015-03-25 58 2015-03-26 69 2015-03-27 65 2015-03-28 60 2015-03-29 60 2015-03-30 64 2015-03-31 55 2015-04-01 55 2015-04-02 56 2015-04-03 52 2015-04-04 55 2015-04-05 62 2015-04-06 57 2015-04-07 58 2015-04-08 63 2015-04-09 63 2015-04-10 57 2015-04-11 53 2015-04-12 56 2015-04-13 53 2015-04-14 53 2015-04-15 57 2015-04-16 64 2015-04-17 66 2015-04-18 66 2015-04-19 70 2015-04-20 73 2015-04-21 63 2015-04-22 60 2015-04-23 54 2015-04-24 54 2015-04-25 56 2015-04-26 60 2015-04-27 77 2015-04-28 60 2015-04-29 61 2015-04-30 63 2015-05-01 65 2015-05-02 65 2015-05-03 69 2015-05-04 63 2015-05-05 58 2015-05-06 62 2015-05-07 69 2015-05-08 75 2015-05-09 80 2015-05-10 67 2015-05-11 57 2015-05-12 60 2015-05-13 54 2015-05-14 64 2015-05-15 68 2015-05-16 60 2015-05-17 67 2015-05-18 78 2015-05-19 71 2015-05-20 74 2015-05-21 78 2015-05-22 62 2015-05-23 61 2015-05-24 64 2015-05-25 60 2015-05-26 71 2015-05-27 76 2015-05-28 82 2015-05-29 79 2015-05-30 73 2015-05-31 77 2015-06-01 61 2015-06-02 64 2015-06-03 68 2015-06-04 73 2015-06-05 80 2015-06-06 85 2015-06-07 88 2015-06-08 87 2015-06-09 84 2015-06-10 78 2015-06-11 76 2015-06-12 68 2015-06-13 75 2015-06-14 82 2015-06-15 86 2015-06-16 73 2015-06-17 77 2015-06-18 76 2015-06-19 75 2015-06-20 77 2015-06-21 78 2015-06-22 77 2015-06-23 79 2015-06-24 78 2015-06-25 87 2015-06-26 89 2015-06-27 92 2015-06-28 83 2015-06-29 84 2015-06-30 87 2015-07-01 90 2015-07-02 93 2015-07-03 92 2015-07-04 92 2015-07-05 91 2015-07-06 85 2015-07-07 81 2015-07-08 86 2015-07-09 84 2015-07-10 70 2015-07-11 72 2015-07-12 79 2015-07-13 78 2015-07-14 82 2015-07-15 79 2015-07-16 79 2015-07-17 82 2015-07-18 92 2015-07-19 95 2015-07-20 80 2015-07-21 75 2015-07-22 75 2015-07-23 79 2015-07-24 73 2015-07-25 70 2015-07-26 72 2015-07-27 74 2015-07-28 82 2015-07-29 90 2015-07-30 94 2015-07-31 94 2015-08-01 92 2015-08-02 87 2015-08-03 83 2015-08-04 79 2015-08-05 74 2015-08-06 77 2015-08-07 83 2015-08-08 77 2015-08-09 83 2015-08-10 84 2015-08-11 86 2015-08-12 83 2015-08-13 83 2015-08-14 65 2015-08-15 71 2015-08-16 77 2015-08-17 81 2015-08-18 86 2015-08-19 89 2015-08-20 73 2015-08-21 72 2015-08-22 80 2015-08-23 82 2015-08-24 75 2015-08-25 78 2015-08-26 83 2015-08-27 85 2015-08-28 74 2015-08-29 72 2015-08-30 68 2015-08-31 66 2015-09-01 67 2015-09-02 67 2015-09-03 65 2015-09-04 65 2015-09-05 69 2015-09-06 61 2015-09-07 70 2015-09-08 73 2015-09-09 76 2015-09-10 77 2015-09-11 81 2015-09-12 80 2015-09-13 69 2015-09-14 62 2015-09-15 64 2015-09-16 68 2015-09-17 65 2015-09-18 67 2015-09-19 70 2015-09-20 73 2015-09-21 65 2015-09-22 66 2015-09-23 69 2015-09-24 72 2015-09-25 60 2015-09-26 65 2015-09-27 64 2015-09-28 70 2015-09-29 71 2015-09-30 65 2015-10-01 70 2015-10-02 60 2015-10-03 67 2015-10-04 73 2015-10-05 74 2015-10-06 65 2015-10-07 61 2015-10-08 66 2015-10-09 67 2015-10-10 70 2015-10-11 64 2015-10-12 65 

In [0]:
weather.rdd.collect()

Out[18]: [Row(date='2015-01-01', temp='42'),
 Row(date='2015-01-02', temp='42'),
 Row(date='2015-01-03', temp='41'),
 Row(date='2015-01-04', temp='51'),
 Row(date='2015-01-05', temp='54'),
 Row(date='2015-01-06', temp='54'),
 Row(date='2015-01-07', temp='46'),
 Row(date='2015-01-08', temp='46'),
 Row(date='2015-01-09', temp='50'),
 Row(date='2015-01-10', temp='46'),
 Row(date='2015-01-11', temp='49'),
 Row(date='2015-01-12', temp='52'),
 Row(date='2015-01-13', temp='49'),
 Row(date='2015-01-14', temp='43'),
 Row(date='2015-01-15', temp='46'),
 Row(date='2015-01-16', temp='53'),
 Row(date='2015-01-17', temp='56'),
 Row(date='2015-01-18', temp='57'),
 Row(date='2015-01-19', temp='50'),
 Row(date='2015-01-20', temp='50'),
 Row(date='2015-01-21', temp='45'),
 Row(date='2015-01-22', temp='49'),
 Row(date='2015-01-23', temp='54'),
 Row(date='2015-01-24', temp='58'),
 Row(date='2015-01-25', temp='63'),
 Row(date='2015-01-26', temp='61'),
 Row(date='2015-01-27', temp='52'),
 Row(date='2015-01-

In [0]:
# Plus grosse temperature par mois (en Celsius :) )
celcius_rdd = weather.rdd.map(lambda x: (x[0], round((int(x[1]) - 32) / 1.8)))
key_value_rdd = celcius_rdd.map(lambda x: (x[0][:7], x[1]))
max_temp_per_month = key_value_rdd.reduceByKey(lambda x, y: max(x, y))
key_value_rdd.collect()

Out[26]: [('2015-01', 6),
 ('2015-01', 6),
 ('2015-01', 5),
 ('2015-01', 11),
 ('2015-01', 12),
 ('2015-01', 12),
 ('2015-01', 8),
 ('2015-01', 8),
 ('2015-01', 10),
 ('2015-01', 8),
 ('2015-01', 9),
 ('2015-01', 11),
 ('2015-01', 9),
 ('2015-01', 6),
 ('2015-01', 8),
 ('2015-01', 12),
 ('2015-01', 13),
 ('2015-01', 14),
 ('2015-01', 10),
 ('2015-01', 10),
 ('2015-01', 7),
 ('2015-01', 9),
 ('2015-01', 12),
 ('2015-01', 14),
 ('2015-01', 17),
 ('2015-01', 16),
 ('2015-01', 11),
 ('2015-01', 12),
 ('2015-01', 12),
 ('2015-01', 8),
 ('2015-01', 7),
 ('2015-02', 9),
 ('2015-02', 11),
 ('2015-02', 10),
 ('2015-02', 11),
 ('2015-02', 13),
 ('2015-02', 14),
 ('2015-02', 12),
 ('2015-02', 15),
 ('2015-02', 13),
 ('2015-02', 13),
 ('2015-02', 13),
 ('2015-02', 17),
 ('2015-02', 16),
 ('2015-02', 14),
 ('2015-02', 12),
 ('2015-02', 15),
 ('2015-02', 16),
 ('2015-02', 12),
 ('2015-02', 11),
 ('2015-02', 11),
 ('2015-02', 12),
 ('2015-02', 12),
 ('2015-02', 13),
 ('2015-02', 11),
 ('2015-02', 10)

In [0]:
celcius_rdd = weather.rdd.map(lambda x: (x[0], round((int(x[1]) - 32) / 1.8)))
celcius_rdd.collect()

Out[19]: [('2015-01-01', 6),
 ('2015-01-02', 6),
 ('2015-01-03', 5),
 ('2015-01-04', 11),
 ('2015-01-05', 12),
 ('2015-01-06', 12),
 ('2015-01-07', 8),
 ('2015-01-08', 8),
 ('2015-01-09', 10),
 ('2015-01-10', 8),
 ('2015-01-11', 9),
 ('2015-01-12', 11),
 ('2015-01-13', 9),
 ('2015-01-14', 6),
 ('2015-01-15', 8),
 ('2015-01-16', 12),
 ('2015-01-17', 13),
 ('2015-01-18', 14),
 ('2015-01-19', 10),
 ('2015-01-20', 10),
 ('2015-01-21', 7),
 ('2015-01-22', 9),
 ('2015-01-23', 12),
 ('2015-01-24', 14),
 ('2015-01-25', 17),
 ('2015-01-26', 16),
 ('2015-01-27', 11),
 ('2015-01-28', 12),
 ('2015-01-29', 12),
 ('2015-01-30', 8),
 ('2015-01-31', 7),
 ('2015-02-01', 9),
 ('2015-02-02', 11),
 ('2015-02-03', 10),
 ('2015-02-04', 11),
 ('2015-02-05', 13),
 ('2015-02-06', 14),
 ('2015-02-07', 12),
 ('2015-02-08', 15),
 ('2015-02-09', 13),
 ('2015-02-10', 13),
 ('2015-02-11', 13),
 ('2015-02-12', 17),
 ('2015-02-13', 16),
 ('2015-02-14', 14),
 ('2015-02-15', 12),
 ('2015-02-16', 15),
 ('2015-02-17', 16)

In [0]:
key_value_rdd = celcius_rdd.map(lambda x: (x[0][:7], x[1]))
key_value_rdd.collect()

Out[23]: [('2015-01', 6),
 ('2015-01', 6),
 ('2015-01', 5),
 ('2015-01', 11),
 ('2015-01', 12),
 ('2015-01', 12),
 ('2015-01', 8),
 ('2015-01', 8),
 ('2015-01', 10),
 ('2015-01', 8),
 ('2015-01', 9),
 ('2015-01', 11),
 ('2015-01', 9),
 ('2015-01', 6),
 ('2015-01', 8),
 ('2015-01', 12),
 ('2015-01', 13),
 ('2015-01', 14),
 ('2015-01', 10),
 ('2015-01', 10),
 ('2015-01', 7),
 ('2015-01', 9),
 ('2015-01', 12),
 ('2015-01', 14),
 ('2015-01', 17),
 ('2015-01', 16),
 ('2015-01', 11),
 ('2015-01', 12),
 ('2015-01', 12),
 ('2015-01', 8),
 ('2015-01', 7),
 ('2015-02', 9),
 ('2015-02', 11),
 ('2015-02', 10),
 ('2015-02', 11),
 ('2015-02', 13),
 ('2015-02', 14),
 ('2015-02', 12),
 ('2015-02', 15),
 ('2015-02', 13),
 ('2015-02', 13),
 ('2015-02', 13),
 ('2015-02', 17),
 ('2015-02', 16),
 ('2015-02', 14),
 ('2015-02', 12),
 ('2015-02', 15),
 ('2015-02', 16),
 ('2015-02', 12),
 ('2015-02', 11),
 ('2015-02', 11),
 ('2015-02', 12),
 ('2015-02', 12),
 ('2015-02', 13),
 ('2015-02', 11),
 ('2015-02', 10)

In [0]:
max_temp_per_month = key_value_rdd.reduceByKey(lambda x, y: max(x, y))
max_temp_per_month.collect()

Out[37]: [('2015-01', '63'),
 ('2015-02', '62'),
 ('2015-03', '69'),
 ('2015-04', '77'),
 ('2015-05', '82'),
 ('2015-06', '92'),
 ('2015-07', '95'),
 ('2015-08', '92'),
 ('2015-09', '81'),
 ('2015-10', '74'),
 ('2015-11', '60'),
 ('2015-12', '60'),
 ('2016-01', '58'),
 ('2016-02', '63'),
 ('2016-03', '71'),
 ('2016-04', '89'),
 ('2016-05', '87'),
 ('2016-06', '93'),
 ('2016-07', '89'),
 ('2016-08', '95'),
 ('2016-09', '78'),
 ('2016-10', '67'),
 ('2016-11', '70'),
 ('2016-12', '50'),
 ('2017-01', '54'),
 ('2017-02', '58'),
 ('2017-03', '58'),
 ('2017-04', '66'),
 ('2017-05', '86'),
 ('2017-06', '96'),
 ('2017-07', '87'),
 ('2017-08', '94'),
 ('2017-09', '90'),
 ('2017-10', '69'),
 ('2017-11', '68'),
 ('2017-12', '54'),
 ('2018-01', '64'),
 ('2018-02', '55'),
 ('2018-03', '73'),
 ('2018-04', '82'),
 ('2018-05', '88'),
 ('2018-06', '88'),
 ('2018-07', '94'),
 ('2018-08', '94'),
 ('2018-09', '85')]